In [1]:
import pandas as pd
import ib_insync as ib 
import nest_asyncio
from datetime import datetime
nest_asyncio.apply()

In [2]:
### Fonction qui te donne l'ensemble de tes pos de IBKR ?
### comment on trade en IBKR ? 
### Mettre en prod le simulated paper trading pour Crude Oil 

### -> tester nos deux backtest via crude oil (vs QuantConnect)
### -> 

In [4]:
from TrendFollowCrudeOil import BaseSignal, CrudeOilTFSignal

def SimulateSignal(
        signal: BaseSignal, 
        date_range: list[str | datetime | pd.Timestamp] | pd.DatetimeIndex,
        ib_client: ib.IB
    ) -> pd.Series:

    assert ib_client.isConnected(), 'Failed to connect to IB'    
    return pd.Series({
        date: signal.get_value(date) 
        for date in date_range
    })

with ib.IB() as ib_client:
    ib_client.connect('127.0.0.1', 7497, clientId=1)
    assert ib_client.isConnected(), 'Failed to connect to IB'
    signal = CrudeOilTFSignal(ib_client)
    values = SimulateSignal(
        signal, 
        date_range=pd.date_range(start='2025-05-01', end=datetime.today()),
        ib_client=ib_client
    )

values

2025-05-01   -1.009409
2025-05-02   -0.994476
2025-05-03   -1.027478
2025-05-04   -1.027478
2025-05-05   -1.027478
2025-05-06   -1.106748
2025-05-07   -0.917563
2025-05-08   -0.910035
2025-05-09   -0.679580
2025-05-10   -0.416445
2025-05-11   -0.416445
2025-05-12   -0.416445
2025-05-13   -0.090707
2025-05-14    0.388898
2025-05-15    0.622489
2025-05-16    0.568912
2025-05-17    0.676373
2025-05-18    0.676373
2025-05-19    0.676373
2025-05-20    0.673181
2025-05-21    0.649722
2025-05-22    0.513822
2025-05-23    0.341878
2025-05-24    0.309462
2025-05-25    0.309462
2025-05-26    0.309462
2025-05-27    0.309462
2025-05-28    0.174282
2025-05-29    0.281272
2025-05-30    0.006802
2025-05-31   -0.277644
2025-06-01   -0.277644
2025-06-02   -0.389476
2025-06-03    0.159610
2025-06-04    0.588640
2025-06-05    0.603489
2025-06-06    0.730918
2025-06-07    1.124836
2025-06-08    1.124836
2025-06-09    1.124836
dtype: float64

In [ ]:
from TrendFollowCrudeOil import BaseSignal
import time

def get_current_position_on_contract(
        all_positions: list[ib.Position], contract: ib.Future
    ) -> int:

    for pos in all_positions:
        if (
            pos.contract.symbol == contract.symbol 
            and 
            pos.contract.lastTradeDateOrContractMonth == contract.lastTradeDateOrContractMonth
            ):
            return pos.position
    return 0

def place_order(qtity_to_buy: int, contract: ib.Future, timeout:float = 15) -> None:

    ### UNTESTED. I DON'T THINK IT WORKS.
    
    if qtity_to_buy == 0:
        return
    elif qtity_to_buy > 0:
        order = ib.MarketOrder('BUY', qtity_to_buy)
    elif qtity_to_buy < 0:
        order = ib.MarketOrder('SELL', -qtity_to_buy)
    print(f"Placing order for {qtity_to_buy} of {contract}")
    trade = ib_client.placeOrder(contract, order)

    start_time = time.time()
    while not trade.isDone() and (time.time() - start_time) <= timeout:
        time.sleep(0.1)
    raise TimeoutError(
            f"Timeout {timeout} while waiting for fill of:\n {trade}"
        )

def ExecuteSignal(signal: BaseSignal, ib_client: ib.IB, multiplier: int = 1) -> None:
        assert ib_client.isConnected(), 'Failed to connect to IB'

        today = pd.Timestamp.now()
        target_position = signal.get_value(today) * multiplier

        current_position = get_current_position_on_contract(
            ib_client.positions(), signal.contract
        )

        place_order(
            qtity_to_buy = int(target_position - current_position),
            contract = signal.contract
        )

        final_position = get_current_position_on_contract(
            ib_client.positions(), signal.contract
        )
        assert (
            final_position == int(target_position)
        ), f'Position not updated correctly. Expected: {target_position}, Got: {final_position}'

        return final_position

with ib.IB() as ib_client:
    ib_client.connect('127.0.0.1', 7497, clientId=1)
    assert ib_client.isConnected(), 'Failed to connect to IB'
    signal = CrudeOilTFSignal(ib_client)
    value = ExecuteSignal(signal, ib_client, multiplier=100)

value 

Placing order for 131 of Contract(secType='FUT', conId=304037495, symbol='CL', lastTradeDateOrContractMonth='20250922', multiplier='1000', exchange='NYMEX', currency='USD', localSymbol='CLV5', tradingClass='CL')


TimeoutError: Timeout 50 while waiting for fill of:
 Trade(contract=Contract(secType='FUT', conId=304037495, symbol='CL', lastTradeDateOrContractMonth='20250922', multiplier='1000', exchange='NYMEX', currency='USD', localSymbol='CLV5', tradingClass='CL'), order=MarketOrder(orderId=40, clientId=1, action='BUY', totalQuantity=131), orderStatus=OrderStatus(orderId=40, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2025, 6, 9, 19, 22, 28, 237666, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')